In [32]:
import torch
import torch.nn as nn

class Net(nn.Module) :
    def __init__(self) :
        super(Net, self).__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 6, 3, 1, 1),
                                nn.BatchNorm2d(6),
                                nn.ReLU(),
                                nn.MaxPool2d(2, 2, 0),
                                
                                nn.Conv2d(6, 12, 3, 1, 1),
                                nn.BatchNorm2d(12),
                                nn.ReLU(),
                                nn.MaxPool2d(2, 2, 0),
                                
                                nn.Linear(8, 4), 
                                nn.Linear(4, 2))
    def forward(self, x) :
        return self.net(x)
net = Net()
# print(net)

# para = list(net.parameters())
# print(para)

input = torch.randn(1, 1, 32, 32)#第一个1表示batch，第二个1表示通道数
output = net(input)
print(output.size())

net.zero_grad()
output.backward(torch.randn(1, 12, 8, 2))



torch.Size([1, 12, 8, 2])


In [28]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

input_size = 5
output_size = 2
batch_size = 30
data_size = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class RandomDataSet(Dataset) :
    def __init__(self, size, length) :
        self.len = length
        self.data = torch.randn(length, size)
    def __getitem__(self, item) :
        return self.data[item]
    def __len__(self) :
        return self.len
    
dataset = RandomDataSet(input_size, data_size) 
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True)

class Model(nn.Module) :
    def __init__(self, input_size, output_size) :
        super(Model, self).__init__()
        self.net = nn.Linear(input_size, output_size)
    def forward(self, x) :
        output = self.net(x) 
        print("\tIn Model: input size", input.size(),
             "output size", output.size())
        return output
    
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
model.to(device)

for data in dataloader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
     "output_size", output.size())

30
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
30
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
30
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
10
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])


In [34]:
import os
import torch
import pandas as pd #用于更容易地进行csv解析
from skimage import io, transform #用于图像的IO和变换
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
%matplotlib
# 忽略警告
import warnings
warnings.filterwarnings("ignore")
plt.ion() # interactive mode

landmarks_frame = pd.read_csv(r'faces/face_landmarks.csv')

# n = 65
# img_name = landmarks_frame.iloc[n, 0]
# # print(img_name)
# landmarks = landmarks_frame.iloc[n, 1 : ]
# landmarks = torch.tensor(landmarks.values.astype('float'))
# landmarks = landmarks.reshape(-1, 2)
# print(landmarks.size())

def show_landmarks(image, landmarks) :
    plt.imshow(image)
    plt.scatter(landmarks[ : , 0], landmarks[ : , 1], s = 10, marker = '.', c = 'r')
    plt.pause(0.001)

# img = io.imread(os.path.join('faces/', img_name))
# show_landmarks(img, landmarks)
# plt.show()

#自定义数据集
class FaceLandmarksDataSet(Dataset) :
    def __init__(self, csv_file, root_dir, transform = None) :
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __getitem__(self, item) :
        img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[item, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[item, 1 : ]
        landmarks = torch.tensor(landmarks.values.astype('float'))
        landmarks = landmarks.reshape(-1, 2)
        sample = {'image' : image, 'landmarks' : landmarks}
        
        if self.transform :
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self) :
        return len(self.landmarks_frame)
face_dataset = FaceLandmarksDataSet(csv_file = r'faces/face_landmarks.csv', root_dir = r'faces/')
#可视化
# for i in range(len(face_dataset)) :
#     sample = face_dataset[i]
#     print(i, sample['image'].shape, sample['landmarks'].shape)
#     h, w = sample['image'].shape[ : 2]
#     print(h, w)
#     ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()#tight_layout会自动调整子图参数，使之填充整个图像区域。这是个实验特性，可能在一些情况下不工作。它仅仅检查坐标轴标签、刻度标签以及标题的部分
#     ax.set_title('Sample #{}'.format(i))
#     ax.axis('off')
#     show_landmarks(sample['image'], sample['landmarks'])
#     if i == 3:
#         plt.show()
#         break

#将样本中的图像重新缩放到给定大小。
# output_size（tuple或int）：所需的输出大小。 如果是元组，则输出为
# 与output_size匹配。 如果是int，则匹配较小的图像边缘到output_size保持纵横比相同
class Rescale() :
    def __init__(self, output_size) :
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        
    def __call__(self, sample) :
        image, landmarks = sample['image'], sample['landmarks']
        h, w = image.shape[ : 2]
        
        if isinstance(self.output_size, int) :
            if h > w :
                new_h, new_w = self.output_size * h / w, self.output_size
            else :
                new_h, new_w = self.output_size, self.output_size * w / h
        else :
            new_h, new_w = self.output_size
            
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        landmarks = landmarks * torch.tensor([new_w / w, new_h / h])#对landmarks的每一行两元素分别乘该行向量
        
        return {'image' : img, 'landmarks' : landmarks}

#随机裁剪样本中的图像.
#output_size（tuple或int）：所需的输出大小。 如果是int，方形裁剪是
class RandomCrop() :
    def __init__(self, output_size) :
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int) :
            self.output_size = (output_size, output_size)
        else :
            assert len(output_size) == 2
            self.output_size = output_size
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)
        
        image = image[top: top + new_h, left: left + new_w]
        landmarks = landmarks - torch.tensor([left, top])
        
        return {'image': image, 'landmarks': landmarks}

#数据增强
class ToTensor() :
    def __call__(self, sample) :
        image, landmarks = sample['image'], sample['landmarks']
        image = image.transpose((2, 0, 1))
        return {'image' : torch.from_numpy(image), 'landmarks' : landmarks}

# scale = Rescale(256)
# crop = RandomCrop(128)
# composed = transforms.Compose([Rescale(256), RandomCrop(224)])
# 在样本上应用上述的每个变换。
# fig = plt.figure()
# sample = face_dataset[65]

# for i, tsfrm in enumerate([scale, crop, composed]):
#     transformed_sample = tsfrm(sample)
# #     print(transformed_sample['image'].shape)
#     ax = plt.subplot(1, 3, i + 1)
#     plt.tight_layout()
#     ax.set_title(type(tsfrm).__name__)
#     show_landmarks(**transformed_sample)
# plt.show()

transformed_dataset = FaceLandmarksDataSet(csv_file = 'faces/face_landmarks.csv', root_dir = 'faces', transform = transforms.Compose([
    Rescale(256),
    RandomCrop(244),
    ToTensor()
]))

# for i in range(len(face_dataset)) :
#     sample = transformed_dataset[i]
#     print(i, sample['image'].size(), sample['landmarks'].size())
#     if i == 3 :
#         break

def show_landmarks_batch(sample_batched) :
    images_batch, landmarks_batch = sample_batched['image'], sample_batched['landmarks']
    batch_size = len(images_batch)
#     print(images_batch.size())
    im_size = images_batch.size(2)
#     print(im_size)
#     print(batch_size)
    #grid_border_size = 2
    #torchvision.utils.make_grid(tensor, padding = 2, nrow = 8)
    #make_grid的作用是将若干幅图像拼成一幅图像。其中padding的作用就是子图像与子图像之间的pad有多宽。
    grid = utils.make_grid(images_batch, padding=0)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size + (i + 1),
                landmarks_batch[i, :, 1].numpy(), s = 10, marker = '.', c = 'r')
    plt.title('Batch from dataloader')
    
dataloader = DataLoader(transformed_dataset, batch_size = 4, shuffle = True, num_workers = 0)

for i_batched, sample_batched in enumerate(dataloader) :
    print(i_batched, sample_batched['image'].size(), sample_batched['landmarks'].size())
    if i_batched == 3 :
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()
        break

Using matplotlib backend: Qt5Agg
0 torch.Size([4, 3, 244, 244]) torch.Size([4, 68, 2])
1 torch.Size([4, 3, 244, 244]) torch.Size([4, 68, 2])
2 torch.Size([4, 3, 244, 244]) torch.Size([4, 68, 2])
3 torch.Size([4, 3, 244, 244]) torch.Size([4, 68, 2])
